# Outlines of Scalable K-means++

## Background

So far, k-means still remains one of the most popular data processing algorithms. It is a widely used technique for statistical data analysis in many fields, such as machine learning, pattern recognition, image analysis and bioinformatics. However, general k-means algorithm with random initialization is not a good clustering algorithm in terms of efficiency and quality, which means it needs a long time to converge when the data set is large and it may just converge to the local optimum. In order to improve the quality, we need to improve the initialization part of the k-means algorithm first, selecting the right centers to do clustering. Recently people have proposed k-means++ initialization algorithm, obtaining the initial centers which can be provably close to the optimal solution, largely improves the quality of k-means algorithm but the problem of inefficiency is still unsolved. Now, there is a new algorithm, k-means||, obtaining a nearly optimal solution after a logarithmic number of passes.

Basically, k-means|| is based on k-means++ and the largest difference between these two algorithm is the initialization part of the algorithm. Since the initialization of k-means++ is deterministic (the previous choices that affect which points are choosed in the current solution), it is nearly impossible to use parallelized computation to improve efficiency. Instead, k-means|| algorithm samples each point independently in each round and repeat the process for approximately O(log $\phi$) rounds, which can be easily implemented by means of parallel computation. Besides, $\phi$ is the cluster cost of initial randomly picked center, which can be viewed as sum of the distances between initial center and other points.


## Algorithm

Pseudo-code

def dist(x,y):
return(distance(x,y))

def cost(C):
return(sum(min(dist(data-C))))

c = sample(data)
phi = cost(c)

for i in range(O(log(phi))):
\begin{equation}
prob = \frac{l*dist(data[i],c)^2}{\phi(c)}
\end{equation}
\begin{equation}
c' = sample(data,prob)
\end{equation}
\begin{equation}
c = [c,c']
\end{equation}

def num_close(c):
minc = argmin([dist(z,y) for y in data for z in c],axis = 1)
return [sum(minc == c) for z in c]

kmeans(c,cluster_number = k)



## Draft of unit test


* Item cost function - non_negativity
* Item cost function - if c has more points cost should be smaller
* Item cost function - if c has all points cost should be 0
* Item probability in sampling is non negative
* Item sum of probability in sampling is l (oversampling factor)
* Item point in C of probability in sampling is 0
* Item find number of closet points function - non negative integer
* Item sum of the weight from weight function should be one
* Item the weight from weight function should be non-negative
* Item total levels of labels of KmeansParallel function should be the same as the number of cluster we want
* Item number of labels should be equal to the number of data



## Optimization - parallel implementation

For the initialization of K-means||, we can use MapReduce model of computation, especially for step 4 and step 7. As to step 4, when we sample each point in the data set in each iteration, we can assign each mapper to sample independently and combine the result. For step 7, we can also assign each mapper to calculate the number of points in data set closer to $c_i$ than any other  potential centers in $C$. In my code, I can use MapReduce model twice for step 7. Basically, I can assign each mapper to find the closest center for a data point independently and then assign each mapper to calculate how many data point is closer to the specific center than other centers independently.

## Data simulation

To generate data which is meaningful for clustering, I simulate some data from a mixture of three bivariate normal distribution.

\begin{eqnarray*}
\begin{pmatrix}x_{1}\\
x_{2}
\end{pmatrix} & \sim & N\left[\left(\begin{array}{c}
3\\
5
\end{array}\right),\left(\begin{array}{ccc}
1 & 0\\
0 & 2
\end{array}\right)\right]\\
\begin{pmatrix}y_{1}\\
y_{2}
\end{pmatrix} & \sim & N\left[\left(\begin{array}{c}
-2\\
3
\end{array}\right),\left(\begin{array}{ccc}
1 & -0.6\\
-0.6 & 1
\end{array}\right)\right]\\
\begin{pmatrix}z_{1}\\
z_{2}
\end{pmatrix} & \sim & N\left[\left(\begin{array}{c}
-6\\
-1
\end{array}\right),\left(\begin{array}{ccc}
3 & 0.3\\
0.3 & 1
\end{array}\right)\right]\\
\end{eqnarray*}



## Algorithm comparison

In order to evaluate the efficiency of the k-means|| algorithm, I compare it with k-means++ and general k-means with simple random initialization. I will try using different sizes of data which are simulated in the same way.